# are discriminated unions like Typescript literal types?

Let us see whether there is an F# equivalent to the following:

```typescript
/**
 * styling variables for this App
 */
export type YouTubeCssVariables =

    /* html anchor */
    '--thumbs-header-link-color' |
    '--thumbs-header-link-text-decoration' |

    /* thumbs set */
    '--thumbs-set-header-color' |
    '--thumbs-set-background-color' |
    '--thumbs-set-header-position' |
    '--thumbs-set-padding-top'
    ;

/**
 * defines a name-value pair for a CSS variable
 */
export interface YouTubeCssOption {
    /**
     * CSS variable name
     */
    variableName: YouTubeCssVariables;

    /**
     * CSS variable value
     */
    variableValue: string;
}
```

With my current level of ignorance, I assume that the F# equivalent is the following type, a discriminated with three members:

In [ ]:
#!fsharp

type YouTubeCssVariable =
    | ThumbsHeaderLinkColor of string
    | ThumbsHeaderLinkTextDecoration of string
    | ThumbsSetHeaderColor of string
    | ThumbsSetBackgroundColor of string
    | ThumbsSetHeaderPosition of string
    | ThumbsSetPaddingTop of string

    member this.Name =
        match this with
        | ThumbsHeaderLinkColor _ -> "--thumbs-header-link-color"
        | ThumbsHeaderLinkTextDecoration _ -> "--thumbs-header-link-text-decoration"
        | ThumbsSetHeaderColor _ -> "--thumbs-set-header-color"
        | ThumbsSetBackgroundColor _ -> "--thumbs-set-background-color"
        | ThumbsSetHeaderPosition _ -> "--thumbs-set-header-position"
        | ThumbsSetPaddingTop _ -> "--thumbs-set-padding-top"

    member this.Value =
        match this with
        | ThumbsHeaderLinkColor v -> v
        | ThumbsHeaderLinkTextDecoration v -> v
        | ThumbsSetHeaderColor v -> v
        | ThumbsSetBackgroundColor v -> v
        | ThumbsSetHeaderPosition v -> v
        | ThumbsSetPaddingTop v -> v

    member this.Pair =
        let n = this.Name
        let v = this.Value
        n, v


This single, `YouTubeCssVariable` type is the equivalent of the [literal type](https://www.typescriptlang.org/docs/handbook/2/everyday-types.html#literal-types), `YouTubeCssVariables` _and_ the interface, `YouTubeCssOption`, we have in Typescript. This means that the equivalent assignment in Typescript:

```typescript
{
    variableName: '--thumbs-set-background-color',
    variableValue: 'inherit'
}
```

…looks like this in F#:

In [ ]:
#!fsharp

let thumbsSetHeaderColor = ThumbsSetBackgroundColor "inherit"

thumbsSetHeaderColor.Pair |> printf "%A"

("--thumbs-set-background-color", "inherit")

@[BryanWilhite](https://twitter.com/BryanWilhite)